In [1]:
# results  = []
# for idx in range(n_rows):
#     row = df.iloc[idx].fillna('')
#     item = {
#         'TITLE': row['Title'],
#         'DATE' : row['Start Date'],
#         'MAINPAGE' : row['Main Page'],
#         'ADDRESS1' : row['Address1'],
#         'ADDRESS2' : row['Address2'],
#         'WEHN' : row['When'],
#         'THUMB': row['thumb'],
#         'IMAGE' : row['image'],
#         'DESC' : row['Desc'],
#         'LongDesc': ''
#     }
#     results.append(item)


In [1]:
#from Levenshtein import ratio, jaro, distance
from collections import Counter

import pandas as pd
#import numpy  as np

import subprocess
import requests
import random
# import gzip
import json
import math
import os
import re

def set_default_path(directory: str | None = 'Event-Aggregator') -> str:
    """Changes the working directory to """
    while not os.getcwd().endswith(directory):
        os.chdir('..')
        if len(os.getcwd()) == 0:
            raise FileExistsError
    return os.getcwd()

def query(url, data, mode):
    match mode:
        case 'POST':
            item = requests.post(url,json=data)
            item = json.loads(item.content.decode())
        case 'GET':
            item = requests.get(url)
            item = json.loads(item.content.decode())
    return item

default_path    = os.path.join(set_default_path() ,'Production_Environment')
server_path     = os.path.join(default_path       ,'Servers') 
collectors_path = os.path.join(server_path        ,'Collectors')

# Events
events_path     = os.path.join(collectors_path    ,'Events')
data_path       = os.path.join(events_path        ,'Data')
File_path       = os.path.join(data_path          ,'Final_Events.csv')

# User Base
UserBase_path  = os.path.join(collectors_path ,'UserBase')
user_data_path = os.path.join(UserBase_path   ,'Data')
user_data_file = os.path.join(user_data_path  ,'User_data.json')

---

In [7]:
[_ for _ in range(10)][:1]

[0]

In [ ]:
url  = "http://127.0.0.1:5000/get/offset"

data = {
    "START"  : "100",
    "OFFSET" : 3 
}
items = query(url = url,data = data, mode = 'POST')
items

[[100,
  'Bollywood Y2K FREE Party : Featuring The World Famous DJ DHARAK',
  'Mar 15',
  'https://pix11.com/calendar/?_escaped_fragment_=/show/?start=2021-03-10#!/details/holi-after-dark-bollywood-neon-glow-party-featuring-india-s-1-dj-dharak/14948564/2025-03-15T21',
  'HK Hall, 605 W 48th St',
  'New York, NY',
  'Sat, Mar 15, 9 PM – Sun, Mar 16, 3 AM',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjVo8ROaSB_G4FpX6E9WVVE9C9DsOK7UdRAN9qCXd-3xHvl61OqQNznLc&s',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQPk4VR0rWG1xVv5xP7onZO5DGF99nZBG4xvBGMcG-n_en_TbfAEX0TaQV6Ag&s=10',
  'Get ready to Glow in the Dark at our Holi After Dark Party - its gonna be lit with Neon Lights and Bollywood Beats by World Famous DJ DHARAK',
  ''],
 [101,
  'Retro Arcade Class w/ NY Society of Play',
  'Mar 11',
  'https://www.hisawyer.com/marketplace/activity-set/1171248',
  'Public School 9 The Sarah Smith Garnet School, 80 Underhill Ave',
  'Brooklyn, NY',
  'Tue, Mar 11, 3 – 4 PM',
  '

In [ ]:
s = "fdsonfonasi"
s.isdigit()

In [ ]:
url  = "http://127.0.0.1:5000/get_rows"
data = {
    'IDS': event_ids
}
event_info = query(url = url,data = data, mode = 'POST')
event_info

In [ ]:
sample_user = list(data.keys())[0]
sample_user

In [ ]:
url  = "http://127.0.0.1:5000/recc"
data = {
    'USER_ID': sample_user,
    'NUMBER': 10,
    'MODE' : 'item'
}
data = query(url = url,data = data, mode = 'POST')
data

---

In [ ]:
import scipy.sparse as sp
import numpy as np

In [ ]:
n_users = len(data.keys())
n_users

In [ ]:
n_events = 489

In [ ]:
mat = sp.dok_matrix((n_users,n_events), dtype=np.int64)
for idx, (user_id, user_data) in enumerate(data.items()):
    u_interact = user_data['CLICKED']
    for event_id, freq in u_interact.items():
        event_id = int(event_id)
        # Add our user and interactions to our mat
        mat[idx,event_id] = freq

In [ ]:

from sklearn.metrics                 import silhouette_score
from sklearn.cluster                 import KMeans

In [ ]:
X = mat.tocsr()

In [ ]:
def user_kmeans(X, n_clusters_max = 15,trials = 5) -> list[float]:
    results = []
    for n_clusters in range(2,n_clusters_max):
        c_score = 0
        for seed in range(trials):
            kmeans = KMeans(
                max_iter     = 100,
                n_clusters   = n_clusters,
                n_init       = 10,
                random_state = seed,
            ).fit(X)
            c_score += silhouette_score(X,kmeans.labels_)

        c_score /= trials
        results.append(float(c_score))
        # cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
        # print(f"Number of elements assigned to each cluster: {cluster_sizes}")
    return results

In [ ]:
results = user_kmeans(X)

In [ ]:
n_clusters = np.argmin(results) + 2
kmeans     = KMeans(
                max_iter     = 100,
                n_clusters   = n_clusters,
                n_init       = 20,
                random_state = 42,
            ).fit(X)

In [ ]:
entry = sp.dok_matrix((1,489), dtype=np.int8)

In [ ]:
label = kmeans.predict(entry).item()

In [ ]:
# entry.toarray()[0][0]